In [4]:
import json

# Read files
with open("../data/DocumentForParsing.txt", "r", encoding="utf-8") as f:
    data1 = json.load(f)

article_text = data1["text"]

with open("../data/DocumentForParsing2.txt", "r", encoding="utf-8") as f:
    data2 = json.load(f)

article_text2 = data2["reference_text"]

print(article_text[:500])
print("\n")
print(article_text2[:500])


As tensions between Donovia and the Western Bloc reach a boiling point, reports of mysterious drone sightings in Temple, Texas, have left residents on edge and sparked concerns about foreign interference in domestic affairs. Central Texas City Grapples with Mysterious Sightings Amidst Growing Tensions The Donovian government has issued a statement denying any involvement, but insiders claim otherwise. The situation is further complicated by the escalating global power struggle between Donovia an


Killeen Cyber Attack Disrupts Local Power Plant. KILLEEN, TX – On the morning of March 26th, the Killeen Power Plant experienced a cyber-attack that cut power to about 2,300 homes in Killeen. Operators detected unusual network activity at 4:00 a.m. and shut off four feeder circuits as a precaution. Workers are facilitating return of power and expect power to return at around 10:00am. What Happened Systems Hit: Main control server and backup communication lines. Impact: Widespread outages in f

In [5]:
# import nltk
# Download NLTK data once
# Vader lexicon used for sentiment analysis
# nltk.download('vader_lexicon')

In [6]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from lexicon import LEXICON as lexicon
sia = SentimentIntensityAnalyzer()

# Keep scores within [-1, 1]
def clamp(score):
    return max(-1.0, min(1.0, score))

# Avoid division by zero if no keyword hits
def safe_div(numer, denom):
    return numer / denom if denom != 0 else 0.0

def analyze_text(text):
    fear = stress = morale = trust = 0
    hits = []
    tokens = text.lower().split()
    max_fear = max_stress = max_morale = max_trust = 0

    for tok in tokens:
        if tok in lexicon:
            f, s, m, t = lexicon[tok]
            fear += f
            stress += s
            morale += m
            trust += t

            # Get max values for scaling
            max_fear += max(0, f)
            max_stress += max(0, s)
            max_morale += max(0, m)
            max_trust += max(0, t)

            hits.append((tok, (f, s, m, t)))

    comp = sia.polarity_scores(text)['compound']
    if comp > 0:
        morale += comp * 1.0
        trust += comp * 0.8
        fear -= abs(comp) * 0.3
        stress -= abs(comp) * 0.5
    elif comp < 0:
        fear += abs(comp) * 0.6
        stress += abs(comp) * 0.6
        morale -= abs(comp) * 0.5
        trust -= abs(comp) * 0.3

    return "fear: " + str(clamp(safe_div(fear, max_fear))) + ", stress: " + str(clamp(safe_div(stress, max_stress))) + ", morale: " + str(clamp(safe_div(morale, max_morale))) + ", trust: " + str(clamp(safe_div(trust, max_trust)))

In [7]:
print("Document 1:", analyze_text(article_text))
print("Document 2:", analyze_text(article_text2))

Document 1: fear: 0.47531052631578946, stress: 0.39354347826086955, morale: 0.7241515151515152, trust: 0.9974249999999999
Document 2: fear: 0.9506185185185185, stress: 0.8977799999999998, morale: -1.0, trust: -0.011119999999999963


In [8]:
test2 = 'Donovia officially invaded America and has taken full control over the country'
test3 = 'America officially invaded Donovia and has taken full control over the country'
test4 = 'The death and destruction is over, the war has ended.'
print("Donovia invades america:", analyze_text(test2))
print("America invades Donovia:", analyze_text(test3))
print("War ends:", analyze_text(test4))

Donovia invades america: fear: 1.0, stress: 1.0, morale: 0.0, trust: 0.0
America invades Donovia: fear: 1.0, stress: 1.0, morale: 0.0, trust: 0.0
War ends: fear: 1.0, stress: 1.0, morale: 0.0, trust: 0.0
